In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

/Users/claudio.rodrigues/Documents/Projects/FanduelHubAI/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Carregar o modelo e tokenizer

In [2]:
MODEL_ID = "meta-llama/Llama-3.2-3B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID
)

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_ID
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.86s/it]


## Configuração prompts e chat

In [3]:
SYSTEM_HEAD = """You are tasked with answering questions regarding the provided documentation. 
Answer ONLY with information found in the documentation.
Provide short and clear answers. Use Markdown to answer all the questions.

Documentation: 

DOCUMENT_TEXT"""

#Substituir aqui com o texto do documento
document_text = "Our new applications currently utilize the updated DevXP Branching strategy, as detailed in the image below. The strategy involves a 'main' branch for creating and deploying releases to Production Environments, a 'staging' branch for merging new code and generating beta releases deployed to Development environments. Additional branches are derived from 'staging' to allow developers to work on their code and approvers to review it before merging. Squash.drawio (1)-20240517-152318.png To accommodate this new strategy, we have set up our Github projects with automation that handles most of the tasks. The details are outlined below. Pull Requests Checks For a developer to merge code into the staging branch, it must go through a Pull Request that requires 2 approvals, one of which must be from a project maintainer. In addition, we have a set of checks that the new code must pass in order to be merged. These checks include Linting, Unit Tests, Typecheck, Integration Tests and, optionally, Contract Tests. We have a reusable workflow available here for quickly setting up a new project while adhering to the same guidelines. These steps ensure that the code adheres to best practices. To close the Pull Request, we use the Squash and Merge option. When we select the Squash and merge option on a pull request, the pull request's commits are squashed into a single commit. Instead of seeing all of a contributor's individual commits from a topic branch, the commits are combined into one commit and merged into the staging branch. Squashing these commits into one commit while merging to the default branch, we retain the original changes with a clear Git history. Also, please take note that the resulting squash commit should adhere to the Commitlint configuration outlined here. Staging Release Once the code has been merged into the staging branch, the next objective is to prepare a beta version for testing by the QA team. To achieve this, it is essential to switch to the staging branch locally and ensure synchronization. Subsequently, execute the following commands: // To create a new tag, verify the latest tag first to ensure the correct version is being used > git tag v1.11.0-beta.1 // Push the new tag > git push origin tag v1.11.0-beta.1 After completing this task, the subsequent step involves initiating a new action to trigger the workflow for generating and uploading the updated RPM to Artifactory. This workflow will also result in the creation of a new release in the repository, as demonstrated in this example. Following this, proceed by triggering a new Jenkins CI Build as usual to obtain the latest version of the RPM. Finally, continue with the deployment process using the same steps as before these modifications. Staging to Main When all the code is merged and deployed to the Development Environment, and successfully passes all QA tests, we are prepared to merge it into main in order to generate a new final release. To proceed with this process, it is necessary to create a new Pull Request and ensure that all checks and approvals described above are met before merging. Traditionally, we utilized the Rebase and Merge option for merging pull requests. By selecting the Rebase and Merge option on a pull request, all commits from the topic branch (or head branch) are integrated into the base branch individually without creating a merge commit. This rebase and merge behavior results in maintaining a linear project history similar to a fast-forward merge. However, this achievement is realized by rewriting the commit history on the base branch with new commits. However, an issue was identified where after merging the pull request, both staging and main branches were consistently out of sync. This occurred due to GitHub's rebase and merge behavior slightly differing from git rebase. When using rebase and merge on GitHub, committer information is always updated resulting in new commit SHAs; whereas outside of GitHub, git rebase does not alter committer information when rebasing onto an ancestor commit. In order to resolve this issue without resorting to manual rebasing, we developed a new Github Action triggered by adding a comment containing /release-to-main text within a Pull Request. This action utilizes the git rebase command to perform a rebase-and-merge operation of  the staging branch into main, ensuring synchronization between both branches at all times. An example of this action can be found here. By adding the comment /release-to-main to the Pull Request, an action will be triggered to rebase and merge the code into main, subsequently closing the Pull Request. Main Release When code is merged to the main branch, we must follow the same process that we followed for the Staging Release, creating and pushing a new tag with the latest version."

user_prompt = "List the steps to create a new tag and push it to the staging branch."

chat = [
    {"role": "system", "content": SYSTEM_HEAD.replace("DOCUMENT_TEXT", document_text)},
    {"role": "user", "content": user_prompt}
]

## Tokenizar o chat

In [4]:
chat_template = tokenizer.apply_chat_template(chat, tokenize=False)

inputs = tokenizer(chat_template, return_tensors="pt")

## Gerar a resposta

In [5]:
outputs = model.generate(
    **inputs, 
    max_new_tokens=128
)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


## Descodificar a resposta

In [6]:
answer = tokenizer.decode(
    outputs[0][inputs['input_ids'].shape[1]:], 
    skip_special_tokens=True
)

In [7]:
answer

'assistant\n\n### Create a new tag and push it to the staging branch\n\n1. **Verify the latest tag**: `git tag v1.11.0-beta.1`\n2. **Push the new tag**: `git push origin tag v1.11.0-beta.1`'